<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.  
(c) Nicole Beisiegel</td>
</table>

In [1]:
%matplotlib inline
import h5py
import os
import types
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython import display
import pylab as pl

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# HOS: A High-Order Spectral Model for 3D Water Wave Simulations
## Visualizing your Output

This ipython notebook contains plotting routines to visualize the HOS model output.

The code can be found on **GitHub**: 
- brennanj/hos (master); 
- beisiegel/hos (fork - current)

Questions can be directed to @beisiegel on Github.

#### Some general remarks on the output files

The model produces output files of the form
- data[xxx].1.h5
- data_extra[xxx].1.h5

with [xxx] $\in [0, \mbox{maxtimestep}]$. Leading zeros are left out. The files contain information on the sea surface elevation and potential velocity.

### `PlotSingleFile(DIR,filename)`

The routine `PlotSingleFile` creates a plot the sea surface elevation $\eta$, the potential $\phi$ and the bathymetry $b$ at a given time step.

Inputs:
- DIR, a string specifying the directory in which the `HDF5`file lies that is to be vizualized
- filename, a string specifying the name of the output file created with the **HOS** model

In [2]:
def PlotSingleFile(DIR, filename):
    f = h5py.File(DIR+fileName, "r")

    # Get the attributes that are contained in the file
    dataset = f.keys()[:]

    #print 'Note: Information contained in this dataset: ' + str(dataset)

    size = f[dataset[5]].shape
    time = f[dataset[dataset.index(u'time')]][0]
    
    # discretization (number of grid cells)
    Nx   = f[dataset[dataset.index(u'Nx')]][0]
    Ny   = f[dataset[dataset.index(u'Ny')]][0]
    
    # total length of the domain
    Lx   = f[dataset[dataset.index(u'Lx')]][0]
    Ly   = f[dataset[dataset.index(u'Ly')]][0]
    
    Kx = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Nx/2,Nx/2-1)])                                                                                                                                                                      
    Ky = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Ny/2,Ny/2-1)])
    
    x = np.asarray([(Lx / Nx) * x for x in np.arange(0,Nx,1)])
    y = np.asarray([(Ly / Ny) * y for y in np.arange(0,Ny,1)])

    X,Y = np.meshgrid(x,y)

    # Plot of the sea surface elevation
    fig = plt.figure()
    fig.set_figwidth(fig.get_figwidth() * 2)
    fig.set_figheight(fig.get_figheight() * 2)

    
    plt.subplot(221, aspect = 'equal')
    plt.contourf(X,Y,f[dataset[5]][:,:], cmap = plt.get_cmap("Blues"))
    plt.xlim(0,Lx)
    plt.ylim(0,Ly)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar()
    plt.title('Sea surface elevation $\eta$ at time t='+str(round(time[0],2)))

    # Plot of the bathymetry
    plt.subplot(223, aspect = 'equal')
    plt.contourf(X,Y,f[dataset[4]][:,:])
    plt.xlim(0,Lx)
    plt.ylim(0,Ly)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.colorbar()
    plt.title('bathymetry $b$ at time t='+str(round(time[0],2)))
    
    # Plot of the velocity potential
    plt.subplot(222, aspect = 'equal')
    
    indx = 6
    if (f.keys()[6] == 'g'): indx = 7
    
    plt.contourf(X,Y,f[dataset[indx]][:,:], cmap = plt.get_cmap("Greens"))
    plt.xlim(0,Lx)
    plt.ylim(0,Ly) 
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Velocity potential $\phi$ at time t='+str(round(time[0],4)))

    plt.colorbar()
    plt.tight_layout()
    plt.show()
    f.close()
    plt.close(fig)

In [3]:
# Example Plot

#DIR      = "/home/nicole/Development/hos/Csource/2dpar/StokesHill/"
#fileName = "data312.1.h5"#

#PlotSingleFile(DIR, fileName)

### `PlotAnimation(DIR,keyword)`

The routine `PlotAnimation` creates an animation of a quantity specifyed through `keyword`. This could either be the sea surface elevation (`keyword = u'eta'`), the potential (`keyword = u'phi'`) or the bathymetry (`keyword = u'bat'`).

Inputs:
- DIR, a string specifying the directory in which the `HDF5`files lay that are to be vizualized
- keyword, a string specifying the quantity of interest (see above)

In [2]:
def PlotAnimation(DIR, keyword):
    idx   = []
    files = []
 
    # Pick and sort all data files
    for file in os.listdir(DIR):
        if (file.startswith('data') & (file[4:7] != '_ex')& (file[4:7] != '_co')):
            idx.append(int(float(file[4:7])))
            files.append(file) 

    files = [x for (y,x) in sorted(zip(idx,files))]

    fig = plt.figure()

    for fileName in files:
        f = h5py.File(DIR+fileName, "r")
        
        # Get the attributes that are contained in the file
        dataset = f.keys()[:]    
        indx_time = dataset.index(u'time')
        
        time = f[dataset[indx_time]][0]
        size = f[dataset[dataset.index(keyword)]].shape
        Lx   = f[dataset[dataset.index(u'Lx')]][0]
        Ly   = f[dataset[dataset.index(u'Ly')]][0]
        
        Nx   = f[dataset[dataset.index(u'Nx')]][0]
        Ny   = f[dataset[dataset.index(u'Ny')]][0]
        
        
        # Create a grid for plotting phi and eta
        xx  = np.linspace(0,Lx,Nx,endpoint=True) 
        yy  = np.linspace(0,Ly,Ny,endpoint=True)
        X,Y = np.meshgrid(xx,yy)
    
        plt.clf()
        plt.contourf(X,Y,f[dataset[dataset.index(keyword)]][:,:], cmap = plt.get_cmap("Blues")) 
        plt.xlim(0,np.amax(xx))
        plt.ylim(0,np.amax(yy))
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Sea surface elevation $\eta$ at time t=' + str(round(time[0],4)))
        plt.colorbar() 
        display.clear_output(wait=True)
        display.display(pl.gcf())
        f.close()
        
    plt.close()
   

In [7]:
# Example Animation

#DIR   = "/home/nicole/Development/hos/Csource/2dpar/"
#keyword = str(u'eta')
#PlotAnimation(DIR,u'eta')

## Statistical Evaluations

### `PlotStatistics(DIR)`

The routine `PlotStatistics` creates a plot of the evolution of skewness and kurtosis over time

Input:
- DIR, a string specifying the directory in which the `HDF5`files lay that are to be analyzed

In [6]:
def PlotStatistics(DIR):
    
    iBound = 5
    idx    = []
    files  = []
    
    for file in os.listdir(DIR):
        if (file.startswith('data') & (file[4:7] != '_ex') & (file[4:7] != '_co')):
            idx.append(int(float(file[4:7])))
            files.append(file) #(file[4:12])

    files = [x for (y,x) in sorted(zip(idx,files))]

    N = len(files)
    
    # Initialize vectors
    kurt  = [] #np.zeros(1,N)
    skew  = [] #np.zeros(1,N)
    t     = [] #np.zeros(1,N)
        
    
    for i, fileName in enumerate(files):
        f = h5py.File(DIR+fileName, "r")

        # read in all the data
        dataset = f.keys()[:]
        
        time = f[dataset[dataset.index(u'time')]][0] 
        eta  = f[dataset[dataset.index(u'eta')]][:,:] 
        phi  = f[dataset[dataset.index(u'phi')]][:,:] 
        Lx   = f[dataset[dataset.index(u'Lx')]][0] 
        Ly   = f[dataset[dataset.index(u'Ly')]][0] 
        Nx   = int(f[dataset[dataset.index(u'Nx')]][0][0])
        Ny   = int(f[dataset[dataset.index(u'Nx')]][0][0])
        g    = 9.81 #adjust to parallel version!!
    
        edge  = np.arange(-iBound,iBound,0.01)
        gauss = np.exp(np.asarray([-.5*x**2 for x in edge]))
   
        Kx = np.asarray([2 * np.pi / Lx * x for x in np.arange(-Nx / 2,Nx / 2)])
        Ky = np.asarray([2 * np.pi / Ly * y for y in np.arange(-Ny / 2,Ny / 2)])
        x  = np.asarray([Lx / Nx * a for a in np.arange(0,Nx)])
        y  = np.asarray([Ly / Ny * b for b in np.arange(0,Ny)])

        eta_vec = eta.reshape((1,Nx * Ny))

        t.append(i) 

        mu3     = np.mean(np.power(np.asarray([ ev - np.mean(eta_vec) for ev in eta_vec]),3))
        mu4     = np.mean(np.power(np.asarray([ ev - np.mean(eta_vec) for ev in eta_vec]),4))
        
        # compute kurtosis and skewness of the sea surface displacement eta.
        skew.append(mu3/np.power(np.std(eta_vec),3))
        kurt.append(mu4/np.power(np.std(eta_vec),4))

        # normalization
        eta_vec = (eta_vec - np.mean(eta_vec))/np.std(eta_vec);
        pdf,pdf_edges = np.histogram(eta_vec,edge)
        
        norm_g  = np.asarray([ gs * np.max(pdf) for gs in gauss])
           
        plt.figure(1)
        plt.gca().set_yscale('log')
        plt.plot(edge,norm_g,'r',linewidth=2)
        #hold on
        plt.plot(edge[:len(pdf)],pdf.T,'b')
        
        
        plt.xlim(-5,5)
        plt.ylim(.5,10**4)
        plt.title ('$\eta$ probabilty density function') 
        #legend('Normal Distribution','\eta PDF','location','NorthWest')
        plt.legend()
        plt.draw()

    tvec = np.asarray([elmt * 60. for elmt in t])

    plt.figure(2)
    plt.plot(tvec,kurt,'bd-');
    plt.title ('Kurtosis evolution') 
    plt.xlabel ('time (s)') 
    plt.ylabel ('kurtosis') 

    plt.figure(3)
    plt.plot(tvec,skew,'ro-');
    plt.title ('Skewness evolution') 
    plt.xlabel ('time (s)') 
    plt.ylabel ('skewness') 

        
    plt.show()
    plt.close()

In [7]:
# Example Statistics

#DIR   = "/home/nicole/Development/hos/Csource/2dpar/" #testsuite_2016-03-01/test4thOrder/"
#PlotStatistics(DIR)

### `PlotSpectra(DIR,filename)`
The following routines plot the (directional) spectrum of the sea surface elevation $\eta$.
The routine `PlotSpectra` creates a plot of the directional  spectrum, a line plot of the spectrum,

Inputs:
- DIR, a string specifying the directory in which the `HDF5`file lies that is to be vizualized
- filename, a string specifying the file of interest


In [8]:
import FtoD as aux
import OmegaTheta as aux2

In [9]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [10]:
def PlotSpectra(DIR, filename):
    f = h5py.File(DIR+fileName, "r")

    # Get the attributes that are contained in the file
    dataset = f.keys()[:]

    size = f[dataset[5]].shape
    time = f[dataset[6]][0]
    
    # discretization (number of grid cells)
    Nx   = f[dataset[dataset.index(u'Nx')]][0]
    Ny   = f[dataset[dataset.index(u'Ny')]][0]
    
    # total length of the domain
    Lx   = f[dataset[dataset.index(u'Lx')]][0]
    Ly   = f[dataset[dataset.index(u'Ly')]][0]
    
    time = f[dataset[dataset.index(u'time')]][0] 
    eta  = f[dataset[dataset.index(u'eta')]][:,:] 
    phi  = f[dataset[dataset.index(u'phi')]][:,:] 
    g    = f[dataset[dataset.index(u'g')]][0] #9.8 or 1.0
     
    Kx = np.asarray([x * 2. * np.pi/Lx for x in np.arange(-Nx/2,Nx/2)])                                                                                                                                                                      
    Ky = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Ny/2,Ny/2)])
    
    x = np.asarray([(Lx / Nx) * w for w in np.arange(0,Nx,1)])
    y = np.asarray([(Ly / Ny) * w for w in np.arange(0,Ny,1)])

    X,Y = np.meshgrid(x,y)

    # Plot of the spectra
    fig = plt.figure()
    plt.ioff()

    #transform_eta = np.fft.fftshift(np.fft.fft2(eta.T))
    #transform_phi = np.fft.fftshift(np.fft.fft2(phi.T))
    
    #hetaD = \#FtoD.FtoD(transform_eta, transform_phi, Kx, Ky,g)
    hetaD     = np.fft.fftshift(np.fft.fft2(eta.T)) #aux.FtoD(transform_eta, transform_phi, Kx, Ky,g=g)
    SpectrumD = np.abs(hetaD)
    #SpectrumD = SpectrumD.T                                                                                                                                                                                 
                                                                                                                                                       
    Spectrum = abs(np.fft.fftshift(np.fft.fft2(eta)));                                                                                                                                  
    
    vecKx = np.zeros(np.size(Kx))
    vecKy = np.zeros(np.size(Ky))
    
    for m in np.arange(np.size(Kx)):
        vecKx[m]= Kx[m][0] / 0.033
        vecKy[m]= Ky[m][0] / 0.033
    
    plt.subplot(2,2,1)  
    plt.loglog(Kx,Spectrum.T[int(Ny/2+1),:])  
    plt.xlabel('Frequency (log)')
    plt.ylabel('Spectrum (magnitude)(log)')
    plt.draw()     

    
    plt.subplot(2,2,2)   
    plt.imshow(np.log10(Spectrum.T), extent = [vecKx[0], vecKx[len(vecKx)-1], \
                                      vecKy[0], vecKy[len(vecKy)-1]], aspect=1)
                                                                                                                         
    plt.grid()  
    #plt.xlim([-100,100])
    #plt.ylim([-100,100])
    plt.title('Spectrum (log)')
    plt.colorbar()
    
    plt.subplot(2,2,3, aspect = 'equal')
    omega, theta = aux2.OmegaTheta(vecKx, vecKy, g=g);      
    X,Y = pol2cart(omega, theta)            
    
    #np.reshape(SpectrumD,())
    #plt.contourf(X,Y,SpectrumD) #, aspect = 10)   
    plt.contourf(X, Y, SpectrumD, cmap = 'Greys')
    #plt.axis([-2, 2, -2, 2])  
    #plt.xlim([1.5,6])
    plt.xlabel('kx')
    #plt.ylim([-1.5,1.5])
    plt.xlabel('ky')
    
    
    plt.subplot(2,2,4)
    plt.plot(Kx, SpectrumD[int(Ny/2+1),:])
    plt.title('Lineplot of directional spectrum')
    plt.xlabel('kx')
    plt.xlim(0,np.amax(Kx))
   
    #fig=plt.figure()
    #plt.contourf(X, Y, SpectrumD, cmap = 'Greys')
    #plt.axis([-2, 2, -2, 2]) 
    #plt.title('Directional spectrum (JONSWAP)', fontsize=18)
    #plt.xlim([1.5,6])
    #plt.xlabel('kx')
    #plt.ylim([-1.5,1.5])
    #plt.ylabel('ky')

    #plt.grid()  
    #plt.colorbar()
    
    #plt.tight_layout()
    plt.show()
    
    f.close()
    plt.close(fig)
    

In [11]:
# Example Spectra

#DIR      = "/home/nicole/Development/hos/Csource/2dpar/"

#fileName = "data0.1.h5"
#PlotSpectra(DIR, fileName)

#fileName = "data1.1.h5"
#PlotSpectra(DIR, fileName)

In [12]:
def PlotSpectraEvolution(DIR, fileName, fileName2):
    f = h5py.File(DIR+fileName, "r")
    f2 = h5py.File(DIR+fileName2, "r")

    # Get the attributes that are contained in the file
    dataset = f.keys()[:]
    size    = f[dataset[5]].shape
    
    # discretization (number of grid cells)
    Nx   = f[dataset[dataset.index(u'Nx')]][0]
    Ny   = f[dataset[dataset.index(u'Ny')]][0]
    
    # total length of the domain
    Lx   = f[dataset[dataset.index(u'Lx')]][0]
    Ly   = f[dataset[dataset.index(u'Ly')]][0]
    
    eta  = f[dataset[dataset.index(u'eta')]][:,:] 
    eta2 = f2[dataset[dataset.index(u'eta')]][:,:] 
    g    = f[dataset[dataset.index(u'g')]][0] #9.8 or 1.0
    
    Kx = np.asarray([x * 2. * np.pi/Lx for x in np.arange(-Nx/2,Nx/2)])                                                                                                                                                                      
    Ky = np.asarray([y * 2. * np.pi/Ly for y in np.arange(-Ny/2,Ny/2)])

    X,Y = np.meshgrid(x,y)

    # Plot of the spectra
    fig = plt.figure()
    plt.ioff()

    hetaD     = np.fft.fftshift(np.fft.fft2(eta.T)) 
    SpectrumD = np.abs(hetaD)                                                                                                                                                                  
    
    hetaD2     = np.fft.fftshift(np.fft.fft2(eta2.T)) 
    SpectrumD2 = np.abs(hetaD2)                                                                                                                                                                  
    
    
    vecKx = np.zeros(np.size(Kx))
    vecKy = np.zeros(np.size(Ky))
    
    for m in np.arange(np.size(Kx)):
        vecKx[m]= Kx[m][0] / 0.033
        vecKy[m]= Ky[m][0] / 0.033
    
    omega, theta = aux2.OmegaTheta(vecKx, vecKy, g=g);      
    X,Y = pol2cart(omega, theta)            

    plt.subplot(211)
    plt.contourf(X, Y, SpectrumD, cmap = 'Greys')
    #plt.xlim([0.0,np.amax(vecKx)])
    plt.xlabel('kx')
    plt.xlabel('ky')
    
    plt.subplot(212)
    plt.contourf(X, Y, SpectrumD2, cmap = 'Greys')
    #plt.xlim([0.0,np.amax(vecKx)])
    plt.xlabel('kx')
    plt.xlabel('ky')
    
    plt.show()
    
    f.close()
    plt.close(fig)

In [13]:

#DIR      = "/home/nicole/Development/hos/Csource/2dpar/"

#fileName = "data0.1.h5"
#fileName2 ="data1.1.h5"
#PlotSpectraEvolution(DIR, fileName, fileName2)
